In [1]:
## Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [3]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [4]:
documents

[Document(id_='dd2ceadf-1a28-4ae3-b024-9db1bd4c45ff', embedding=None, metadata={'page_label': '1', 'file_name': 'attention.pdf', 'file_path': 'data\\attention.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-01-29', 'last_modified_date': '2024-01-29', 'last_accessed_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.co

In [5]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

e:\New Recordings\Langchain Videos\LLAmindex\Projects\Llama_index\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 36/36 [00:03<00:00, 11.11it/s]


In [6]:
index

In [7]:
query_engine=index.as_query_engine()

In [22]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])


In [23]:
response=query_engine.query("What is attention is all yopu need?")

In [24]:
from llama_index.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: The paper "Attention Is All You Need" proposes a new
network architecture called the Transformer. This architecture is
based solely on attention mechanisms and does not use recurrent or
convolutional neural networks. The paper demonstrates that the
Transformer models outperform existing models in terms of quality,
parallelizability, and training time. The Transformer achieves state-
of-the-art results in machine translation tasks and generalizes well
to other tasks such as English constituency parsing.
______________________________________________________________________
Source Node 1/1
Node ID: be144ab8-cb0a-44fa-af69-3dbfe555e41a
Similarity: 0.8107415810551661
Text: Provided proper attribution is provided, Google hereby grants
permission to reproduce the tables and figures in this paper solely
for use in journalistic or scholarly works. Attention Is All You Need
Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google
Brain noam@google.comNiki Parmar∗ Goo

In [25]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)

Transformers are a model architecture that rely entirely on an attention mechanism to draw global dependencies between input and output. They eschew recurrence and do not use sequence-aligned RNNs or convolution. Transformers allow for significantly more parallelization and have been shown to achieve state-of-the-art results in tasks such as translation. They can be trained faster than architectures based on recurrent or convolutional layers.
